In [1]:
!pip install "dlt[lancedb]"
!pip install sentence-transformers

In [2]:
!python --version


Python 3.10.12


In [3]:
import requests
import dlt

qa_dataset = requests.get("https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1").json()

@dlt.resource
def qa_documents():
  for course in qa_dataset:
    yield course["documents"]

pipeline = dlt.pipeline(pipeline_name="from_json", destination="lancedb", dataset_name="qanda")

load_info = pipeline.run(qa_documents, table_name="documents")

print(load_info)

/home/azureuser/zaid_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pipeline from_json load step completed in 0.18 seconds
1 load package(s) were loaded to destination LanceDB and into dataset qanda
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x73e57518df90> location to store data
Load package 1722307714.3346643 is LOADED and contains no failed jobs


In [52]:
import lancedb
import pandas as pd
db = lancedb.connect("./.lancedb")
print(db.table_names())

['notion_pages____dlt_loads', 'notion_pages____dlt_pipeline_state', 'notion_pages____dlt_version', 'notion_pages___dltSentinelTable', 'notion_pages___employee_handbook', 'qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents', 'qanda_embedded____dlt_loads', 'qanda_embedded____dlt_pipeline_state', 'qanda_embedded____dlt_version', 'qanda_embedded___dltSentinelTable', 'qanda_embedded___documents']


In [5]:
db_table = db.open_table("qanda___documents")

db_table.to_pandas()

,id__,text,section,question,_dlt_load_id,_dlt_id
0,dc154cc7-2468-57e9-99a1-728724256b7f,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1722282007.5299253,wCmu+5mEXpldow
1,898131a3-763b-5b51-bbca-64c9b1e35bec,GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1722282007.5299253,FpMDak/R/0oA8w
2,1b33adde-252c-5248-ac46-12d44fb7e50a,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1722282007.5299253,rN3Xb39pl/fyHA
3,e0c34fa2-713b-5580-9174-6c78f3acecf8,You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1722282007.5299253,vKsqhMj+uKvasw
4,05087419-2ebc-5b54-a4ed-5757f7b324eb,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1722282007.5299253,S4qpItXt6A1mxQ
...,...,...,...,...,...,...
1891,f423081e-c51d-5c1a-9440-add16b6565da,Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1722307714.3346643,51Cw4BaKdrQ7aQ
1892,7d8429a8-56c3-5042-8b8f-5a9c8191b074,Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1722307714.3346643,5aABvsSzxSVhyA
1893,b38434a2-e4da-5048-b59d-9c082ce4ce01,Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1722307714.3346643,TT6WNOq43mbKwQ
1894,5cafca63-1c1d-56e7-8d4d-8513f26a8a81,Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1722307714.3346643,XCX0hICuXAvCtg


In [40]:
import os
from dlt.destinations.adapters import lancedb_adapter

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

pipeline = dlt.pipeline(pipeline_name="from_json_embedded", destination="lancedb", dataset_name="qanda_embedded")

load_info = pipeline.run(lancedb_adapter(qa_documents, embed=["text", "question"]), table_name="documents")
print(load_info)

Pipeline from_json_embedded load step completed in 4.83 seconds
1 load package(s) were loaded to destination LanceDB and into dataset qanda_embedded
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x73e44bfe0f70> location to store data
Load package 1722312955.4100904 is LOADED and contains no failed jobs


In [41]:
db = lancedb.connect("./.lancedb")
print(db.table_names())

['notion_pages____dlt_loads', 'notion_pages____dlt_pipeline_state', 'notion_pages____dlt_version', 'notion_pages___dltSentinelTable', 'notion_pages___employee_handbook', 'qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents', 'qanda_embedded____dlt_loads', 'qanda_embedded____dlt_pipeline_state', 'qanda_embedded____dlt_version', 'qanda_embedded___dltSentinelTable', 'qanda_embedded___documents']


In [42]:
db_table = db.open_table("qanda_embedded___documents")
db_table.to_pandas()

,id__,vector__,text,section,question,_dlt_load_id,_dlt_id
0,544e9542-5254-58be-91ee-ca5b46a091af,"[-0.00035095983, -0.062014256, -0.03799991, 0....",The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1722283719.3086917,cyjYbx5U1lOKAw
1,9b10d6cf-e16f-5d29-aef5-3d38d5528fe1,"[0.020011423, -0.011535534, 0.0130172055, -0.0...",GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1722283719.3086917,R7mudEjsMDsqLQ
2,30d25db2-d9bb-5e29-a904-cec36e2e4268,"[0.014857549, -0.06664994, -0.013571203, 0.023...","Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1722283719.3086917,hU/t9ABjXn123g
3,db21e45f-7d93-5b6e-b6f8-7d9e6b153f7e,"[-0.023312101, -0.09461491, 0.056361604, -0.00...",You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1722283719.3086917,ciyznGiGe6Iehw
4,3b09cf1e-d1cf-5ccf-a5ca-f8e88b580738,"[0.026537659, -0.017796658, 0.0021156375, 0.00...",You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1722283719.3086917,y9Yb4fFFlAyzpw
...,...,...,...,...,...,...,...
3787,6b3dbd9a-f6af-56db-8c44-7b1876bcefcf,"[0.016619304, -0.033603117, -0.09334721, -0.02...",Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1722312955.4100904,Mu8qTJfFZHHHaw
3788,12c44bbd-b034-5610-9e33-0006afa50c3e,"[0.026872871, -0.0019949335, 0.008369085, -0.0...",Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1722312955.4100904,J9LzfjX1fh6ebw
3789,38a0e3de-3cae-53c1-9430-b73999cd5421,"[0.03513752, 0.056265578, 0.024428517, -0.0651...",Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1722312955.4100904,ElQAXOhIGZOZFw
3790,4e264772-fcc6-57b7-979d-7fde95293445,"[0.03380975, -0.0031219234, 0.0017484598, 0.01...",Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1722312955.4100904,DsOGKAT/LrGt2A


In [43]:
# !yes | dlt init rest_api lancedb

In [44]:
import os


os.environ["SOURCES__REST_API__NOTION__API_KEY"] = 'secret_q4QgENGc9JKXgGContzdV6Z6KMTwzLYXWX9VMFi1dlb'

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

In [45]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

In [59]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="homework")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "workshop",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config, name="homework")


def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="homework",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    last_value = last_edited_time.last_value
    print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        print(block)
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

2024-07-18T17:28:00.000Z
{'block_id': 'dd6469c1-fa1d-42d1-84c2-7e657b1dc47d', 'block_type': 'heading_2', 'content': '', 'last_edited_time': '2024-06-26T09:00:00.000Z', 'inserted_at_time': datetime.datetime(2024, 7, 30, 4, 31, 3, 234318, tzinfo=datetime.timezone.utc)}
{'block_id': 'c0262981-b5f1-4a57-a91f-2e75f649b86c', 'block_type': 'paragraph', 'content': 'Our company operates between 9 a.m. to 7 p.m. on weekdays. You may come to work at any time between 9 a.m. and 11 a.m., depending on your team’s needs. Some departments may work after hours, too (e.g. customer support, shipping.) If you work in these departments, you will follow a shift schedule as needed.', 'last_edited_time': '2024-07-18T14:00:00.000Z', 'inserted_at_time': datetime.datetime(2024, 7, 30, 4, 31, 3, 234643, tzinfo=datetime.timezone.utc)}
{'block_id': 'ab51a1ee-5ca0-4933-b93c-f3e325ad178f', 'block_type': 'heading_2', 'content': '', 'last_edited_time': '2024-06-26T09:01:00.000Z', 'inserted_at_time': datetime.datetime(2

In [57]:
import lancedb

db = lancedb.connect(".lancedb")

print(db.table_names())
    
# notion_pages__homework

['notion_pages____dlt_loads', 'notion_pages____dlt_pipeline_state', 'notion_pages____dlt_version', 'notion_pages___dltSentinelTable', 'notion_pages___employee_handbook', 'notion_pages___homework', 'qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents', 'qanda_embedded____dlt_loads', 'qanda_embedded____dlt_pipeline_state', 'qanda_embedded____dlt_version', 'qanda_embedded___dltSentinelTable', 'qanda_embedded___documents']


In [58]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework")

df = dbtable.to_pandas()
print(len(df))

14


In [60]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


######################################################################## 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> Enabling and starting ollama service...
Created symlink /etc/systemd/system/default.target.wants/ollama.service → /etc/systemd/system/ollama.service.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [61]:
!nohup ollama serve > nohup.out 2>&1 &

OSError: Background processes not supported.

In [62]:
%%capture
!ollama pull llama2-uncensored

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [63]:
!pip install ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 KB 9.4 MB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [64]:
import ollama

In [70]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [71]:
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___homework")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You:how many PTO days are the employees entitled to in a year ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

In [ ]:
main()

You:how many PTO days are the employees entitled to in a year how many PTO days are the employees entitled to in a year
Assistant: Employees in this company are entitled to [20 days] of Paid Time Off (PTO) per year, which can be used for various purposes. Your PTO accrual begins the day you join our company and you receive 1.7 days per month. You can take your PTO at any time after your first week with us and you can use time off you haven’t accrued yet. You will earn one additional day per year after your first year with our company, with a cap at 25 days overall. If you want to use PTO, send a request through the HRIS system. If your manager or HR approves, you are permitted to take your leave.
Employees can also transfer their accrued PTO to the next year if they wish to do so. However, we encourage our employees to use their time off throughout the year instead of saving it up for later.
If you leave this company, you may be entitled to receive compensation for your accrued PTO dep